In [1]:
import os
import json
import shutil
import lzma

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

C:\Users\rdm42\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [84]:
download_dir = os.path.join('..','0_data','downloaded_posts')
metadata_paths = [os.path.join(download_dir,f) for f in os.listdir(download_dir) if f.endswith('.json.xz')]

additional_data = []

for i,path in enumerate(metadata_paths):

    with lzma.open(path, 'rt') as file:
        data_dict = json.load(file)
    
    short = data_dict['node']['shortcode']
    user  = data_dict['node']['owner']['username']
    likes = data_dict['node']['edge_media_preview_like']['count']
    caption = data_dict['node']['edge_media_to_caption']['edges'][0]['node']['text']
    long = os.path.split(path)[-1].split('.')[0]

    additional_data.append((short,user,likes,caption,long))

additional_data = pd.DataFrame(additional_data, columns=['Short','user','likes','caption','Long']) \
                    .drop(columns=['Long'])
additional_data['URL'] = additional_data.Short.apply(lambda code: f"https://www.instagram.com/p/{code}")
additional_data.head(3)

,Short,user,likes,caption,URL
0,fNO52Nxluy,amenstyle,61,Valentina Vignali looks explosive in a preciou...,https://www.instagram.com/p/fNO52Nxluy
1,lzQl_gRlnD,amenstyle,374,Regram @nimabenati #amenstyle #amen #fashion #...,https://www.instagram.com/p/lzQl_gRlnD
2,mrzAb-Rlkt,amenstyle,175,RG @nimabenati #amencouture amenstyle.com,https://www.instagram.com/p/mrzAb-Rlkt


In [80]:
with pd.ExcelFile('pictures_captions_autocaptions.xlsx') as xl:
    autocaptions = xl.parse()[['Short','Auto_caption']]
    
# autocaptions['Long'] = autocaptions.Long.str.extract("(.*_UTC)")[0]

In [87]:
output_table = autocaptions.merge(additional_data, on='Short', how='left', validate='m:1') \
               [['Short','URL','user','likes','caption','Auto_caption']]

In [90]:
output_table.to_clipboard(index=False)

In [97]:
output_table[output_table.duplicated(keep=False)]

,Short,URL,user,likes,caption,Auto_caption
168,CiaXyUYsqcJ,https://www.instagram.com/p/CiaXyUYsqcJ,jacquemus,188018.0,Bella in Jacquemus 🏹🫠,a picture of a woman with long hair wearing a ...
170,CiaXyUYsqcJ,https://www.instagram.com/p/CiaXyUYsqcJ,jacquemus,188018.0,Bella in Jacquemus 🏹🫠,a picture of a woman with long hair wearing a ...


# Agregar obj_rate a LIWC

In [ ]:
import pandas as pd
import os

In [2]:
with pd.ExcelFile("LIWC2015 Results (Ig posts sample - 11012024).xlsx") as xl:
    liwc_data = xl.parse()

C:\Users\rdm42\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [35]:
path = os.path.join('data','resources','database.xlsx')

with pd.ExcelFile(path) as xl:
    original_db = xl.parse(usecols='B:C').dropna()
    
original_db['Short'] = original_db.link.str.split('/').str[4]
original_db.drop_duplicates(subset=['Short'],inplace=True)

In [54]:
df = liwc_data.merge(original_db.drop(columns=['link']).rename(columns={'Objectify (yes or no)':'objectification_flag'}),how='left',on='Short',validate='m:1')

cols = df.columns.tolist()
df[cols[:6] + [cols[-1]] + cols[6:-1]].to_excel('liwc_data_11012024.xlsx',index=False)

<ipython-input-54-f23867d87979>:4: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df[cols[:6] + [cols[-1]] + cols[6:-1]].to_excel('liwc_data_11012024.xlsx',index=False)


In [57]:
df.query("Short=='Cll7fIRsuo2'")

,Short,URL,user,likes,caption,Auto_caption,WC,Analytic,Clout,Authentic,...,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP,objectification_flag
194,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a couple enjoys a peaceful day at...,22,99.0,50.00,61.34,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
195,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a serene beach scene with a white...,24,99.0,50.00,1.79,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
196,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a woman with long brown hair wear...,21,99.0,68.29,68.01,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
197,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a young boy and girl enjoy a day ...,31,99.0,74.07,60.40,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
198,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a woman stands barefoot on a beac...,34,99.0,72.17,1.00,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
199,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a woman with long hair sits on a ...,30,99.0,63.05,10.08,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
200,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a barefoot person stands on a bla...,21,99.0,68.29,13.15,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
201,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a woman with black hair lies on t...,31,99.0,83.34,1.00,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
202,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,"a picture of two women, one with black hair an...",24,99.0,33.83,23.51,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
203,Cll7fIRsuo2,https://www.instagram.com/p/Cll7fIRsuo2,jacquemus,227480.0,« IN BED WITH »\nJACQUEMUS + TEKLA \nJacquemus...,a picture of a woman relaxes on a metal bench ...,25,99.0,65.56,43.37,...,0,0,0,0,0.0,0,0.0,0,0.0,1.0
